In [1]:
import os
from scipy.spatial import distance, KDTree, cKDTree
import math
import numpy as np
import time

In [77]:
def parse_items(input_data):
    
    lines = input_data.strip().split('\n')
    items = []
    
    for i,line in enumerate(lines):
        line = line.split()
        if i==0: 
            #1st line is header
            num_nodes = int(line[0])
        else:
            items.append(tuple(map(float, line)))

    return num_nodes, items

def compute_distance(p1,p2):
    return math.hypot(p1[0]-p2[0],p1[1]-p2[1])

def compute_objective(path, coords):

    path = path+[path[0]]
    obj = 0
    for start,end in zip(path[:-1],path[1:]):
        obj+=compute_distance(coords[start], coords[end])
    
    return obj

data_dir = '../data/'
file_name = 'tsp_51_1'

with open(os.path.join(data_dir, file_name), 'r') as input_data_file:
            input_data = input_data_file.read()
num_nodes, coords = parse_items(input_data)
print('num_nodes =', num_nodes)

num_nodes = 51


In [4]:
# https://stackoverflow.com/questions/13079563/how-does-condensed-distance-matrix-work-pdist

# def square_to_condensed(i, j, n):
#     assert i != j, "no diagonal elements in condensed matrix"
#     if i < j:
#         i, j = j, i
#     return n*j - j*(j+1)//2 + i - 1 - j

# def calc_row_idx(k, n):
#     return int(math.ceil((1/2.) * (- (-8*k + 4 *n**2 -4*n - 7)**0.5 + 2*n -1) - 1))

# def elem_in_i_rows(i, n):
#     return i * (n - 1 - i) + (i*(i + 1))//2

# def calc_col_idx(k, i, n):
#     return int(n - elem_in_i_rows(i + 1, n) + k)

# def condensed_to_square(k, n):
#     i = calc_row_idx(k, n)
#     j = calc_col_idx(k, i, n)
#     return i, j


### Nearest Insertion

In [68]:
# # compute distances
# condensed_distances = distance.pdist(coords, metric='euclidean')
# distances = distance.squareform(condensed_distances)

In [69]:
# # initialize
# n_nodes = len(coords)
# nodes = list(range(n_nodes))

# node = 0
# nodes.remove(node)
# nearest = np.argsort(distances[node,:])[1]
# nodes.remove(nearest)
# path = [node, nearest] 

In [70]:
# while nodes:
#     # select nearest node
#     nearest, min_dist = None, math.inf
#     for n in nodes:
#         for i in path:
#             d = distances[i,n]
#             if d < min_dist:
#                 nearest, min_dist = n, d
#     nodes.remove(nearest)
#     # select insertion edge
#     replaced, min_cost = (None, None), math.inf
#     for i,j in zip(path, path[1:]+[path[0]]):
#         cost = distances[i,nearest] + distances[nearest,j] - distances[i,j]
#         if cost < min_cost:
#             replaced, min_cost = (i,j), cost
#     path.insert(replaced[0]+1, nearest)

### Nearest Neighboor

In [74]:
def tsp_nn(coords, num_nodes):

    # initialize
    coords = np.array(coords)
    nodes = list(range(num_nodes))
    visited = np.zeros(num_nodes,dtype=bool)
    tree = cKDTree(np.array(coords[nodes]))
    # startting node
    nearest = 0
    visited[nearest] = True
    path = [nearest]
    # visit nearest neighbours 
    while not visited.all():
        for k in range(2,num_nodes+1):
            dd, ii = tree.query(coords[nearest], k=k)
            # print(k, nearest, ii, dd, visited[ii])
            if not visited[ii].all():
                for n in ii:
                    if not visited[n]:
                        visited[n] = True
                        nearest = n
                        path += [nearest]
                        # print(f'>>>>> added {nearest} to path')
                        break
                break
    return path

In [78]:
path = tsp_nn(coords, num_nodes)
obj = compute_objective(path,coords)

results = f'{obj} {0}\n{" ".join(map(str, path))}'
with open('solution', 'w') as f:
    f.write(results)

In [76]:
def solver_submission(input_data):

    num_nodes, coords = parse_items(input_data)
    path = tsp_nn(coords, num_nodes)
    obj = compute_objective(path,coords)

    return f'{obj} {0}\n{" ".join(map(str, path))}'

data_dir = '../data/'
file_name = 'tsp_51_1'

with open(os.path.join(data_dir, file_name), 'r') as input_data_file:
            input_data = input_data_file.read()
with open('solution', 'w') as f:
    f.write(solver_submission(input_data))


    